In [1]:
from IPython.display import HTML
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this Jupyter notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
''')

![title](figs/COMPGEOP_TITLE.jpg)

### <h1><center>Module 10: Numerical Solutions to Hyperbolic PDEs</center></h1>

This module focuses on generating numerical solutions of **hyperbolic** partial differential equations (PDEs).  Recall that these types of second-order PDEs

$$Au_{xx}+2Bu_{xy}+Cu_{yy}+Du_{x}+Eu_{y}+Fu+G=0 \tag{1}$$

are defined by the condition $B^2-AC > 0$ (where $y$ is considered the $t$ variable and thus $A=1$, $C=-\frac{1}{c}$ and all others are equal 0).

Three interesting examples (with increasing complexity) are the following:

   * **Acoustic wave equation (Homogeneous medium)**: The acoustic wave equation (AWE) is normally used to model the pressure disturbance $\phi=\phi(x,y,z,t)$ generated by an acoustic wave propagation through a fluid medium defined by constant velocity $v=\sqrt{\frac{K}{\rho}}$ where $K$ is the bulk compressibility and $\rho$ is the density.
   
$$\frac{\partial^2 \phi}{\partial t^2} - v^2\nabla^2 \phi = F(x,y,z,t) \tag{2a}$$ 

where $F(x,y,z,t)$ is an external force term. Because this is a second-order temporal partial derivative, to uniquely define the wavefield solution one needs to define **two initial conditions** $\phi(x,y,z,t=0)=f(x,y,z)$ and $\frac{\partial \phi}{\partial t}(x,y,z,t=0)=\phi_t=g(x,y,z)$. (Note that the subscript indicates which partial derivative has been applied.) One also has to prescribe the behavior of the solution on the domain boundary $D$ (e.g., $\left. \phi\right|_{\partial D}=B(x,y,z)$). Note that the acoustic wave equation is commonly used in exploration seismology to approximate compressional (i.e., P-wave) propagation through complex heterogeneous media.
   
   * **Acoustic wave equation (variable density)**: This is like the example above, except now that the density is assumed to spatially vary:

$$\frac{\partial^2 \phi}{\partial t^2}  -  K \nabla \cdot \left(\frac{1}{\rho} \nabla \phi  \right) = F(x,y,z,t). \tag{3b}$$ 
   
This equation is commonly used in computational ocean acoustics because it allows one to introduce variations in density caused by temperature and salinity concentration among others.  This wave equation requires the same type of initial and boundary conditions as the homogenous equation above.


   * **Advection**:  We have actually looked at this hyperbolic equation in Module 7 of the course. One can recognize this to be the case if we **factorize** the 1D AWE given above in the following way
   
$$\left[\frac{\partial^2 }{\partial t^2} - v^2\frac{\partial}{\partial x^2} \right] \phi = 
\left(\frac{\partial }{\partial t}-v \frac{\partial}{\partial x} \right)\left( \frac{\partial }{\partial t}+v \frac{\partial}{\partial x}\right) \phi = F(x,t). \tag{3c}$$ 
 
Here we see that we have the produce of the left- and right-going advection equations. This was recognized in 1747 by French scientist [d'Alembert](https://en.wikipedia.org/wiki/D%27Alembert%27s_formula).  Note that one sometimes see the **d'Alembert operator** $\Box^2$ in various problems in mathematical physics, which is given by:

$$\Box^2 \phi = \left[\frac{\partial^2 }{\partial t^2} - v^2\frac{\partial}{\partial x^2} \right] \phi\tag{3d}$$


In the section below, we will look at generating solutions $\phi$ of the 1D and 2D wave equation throughout a computational domain $D$.  We will also examine different boundary conditions (BCs) on boundary of D (defined by shortform $\delta \Omega$), namely the Dirchelet BC

$$ \phi(\mathbf {x}) = f(\mathbf {x}), \quad \forall \mathbf {x}\in \delta \Omega \tag{4a}$$

and the Neumann BC

$${\frac {\partial \phi}{\partial \mathbf {n} }}(\mathbf {x} )=f(\mathbf {x} )\quad \forall \mathbf {x} \in \partial \Omega \tag{4b}$$

where ${\bf n}$ is a normal vector defined at the grid boundary that points inward.

## 1D Acoustic wave equation

Let's first look at the problem of an acoustic wave propagating on a homogeneous taut string of length $L$ with fixed ends. In this case, we can assume that there is no external forcing function $F(x,t)$ and the string starts from some combination of an initial displacement disturbance $\phi(x,t=0) = f(x)$ and an initial velocity $\phi_t(x,t=0) = g(x)$. Thus, the PDE we are solving is

$$\frac{\partial^2 \phi}{\partial t^2} = v^2\frac{\partial^2 \phi}{\partial x^2}. \tag{5}$$ 

Let's now look at an **explicit** discretization method to solve equation 5 (as opposed to the **implicit** ones like Crank-Nicolson approach we studied in Module 9).  Let's start by using a standard $\mathcal{O}(\Delta x^2,\Delta t^2)$ discretization.

$$\frac{\phi^{n+1}_i - 2\phi^{n}_i +\phi^{n-1}_i}{\Delta t^2} = 
v^2 \left( \frac{\phi^{n}_{i+i} - 2\phi^{n}_i +\phi^{n}_{i-1}}{\Delta x^2} \right) \tag{6}$$

Let's now multiply through by $\Delta t^2$ and use the square of the Courant number $C^2 = \frac{v^2\Delta t^2}{\Delta x^2}$ from the previous modules to write

$$\phi^{n+1}_i - 2\phi^{n}_i +\phi^{n-1}_i = C^2 \left( \phi^{n}_{i+i} - 2\phi^{n}_i +\phi^{n}_{i-1}  \right). \tag{7}$$

Rearranging terms to isolate the unknown quantity at time step $n+1$ on the left-hand side and keep known quantities on the right-hand side leads to:

$$\phi^{n+1}_i = 2\phi^{n}_i -\phi^{n-1}_i +C^2 \left(\phi^{n}_{i+i} -2 \phi^{n}_i + \phi^{n}_{i-1}\right). \tag{8}$$

### Initial Conditions

Above we noted that two initial conditions are required for this PDE because of the second temporal derivative. However, in the above discretization it is not apparent how the derivative initial conditions can be introduced.  One thing to note is that as part of the discretization we now have the term $\phi^{n-1}_i$ which represents the wavefield at the previous time step. Thus, this can be used to set the second condition.

$$\frac{\partial \phi}{\partial t} \approx \frac{\phi^n_i - \phi^{n-1}_i}{\Delta t} = g(x)$$

Thus, we can specify the previous time step as

$$\phi^{n-1}_i = \phi^n_i-\Delta t g(x)$$


### Handling Boundaries with Ghost Points

We can also explore what happens at the left and right boundaries from equation 8.  First, at the left hand boundary (i.e., $i=0$) we can write

$$\phi^{n+1}_0 = 2\phi^{n}_0 -\phi^{n-1}_0 +C^2 \left(\phi^{n}_{1} -2 \phi^{n}_0 + \phi^{n}_{-1}\right). \tag{8}$$

Note that there is a $ \phi^{n}_{-1}$ term in this equation, which represents a point out of the grid! Let's assume that we can set $ \phi^{n}_{-1}=0$, which is kind of like using a **ghost point** which isn't formally defined, but we conjured it up for numerical expediency.  Similarly, on the right-hand side we will have to do the same at ghost point $ \phi^{n}_{N+1}=0$.

Let's first build a 1D AWE solver:

In [11]:
def awe_explicit_solver_homogeneous_2nd_order(Wo,Wm,dx,dt,v,LB,RB):
    '''Set up second-order solver of the acoustic wave equation
    usage: U = awe_explicit_solver_homogeneous(Wo,Wm,dx,dt,v,LB,RB)
    input: 
        Wo: Acoustic pressure vector (nx) at time step n
        Wm: Acoustic pressure vector (nx) at time step n-1
        dx: Spatial sampling
        dt: Temporal sampling
        v : Homogeneous propagation velocity 
        LB: Left  boundary (Homogeneous Dirchelet)
        RB: Right boundary (Homogeneous Dirchelet)
    output:
        Wm: Acoustic pressure vector (nx) at time n+1
    dependencies:
        None
    written by Jeff Shragge, jshragge@mines.edu, 10/2019
    '''        
    ## . . Get dimensions of wavefield
    nx = len(Wo)
    
    ## . . Define Courant number (squared)
    C2 = (v*dt/dx)**2
        
    ## . . Update solution 
    ## . . Note that I am overwriting Wm because this wavefield
    ## . . isn't needed after this calculation!
    Wm[1:nx-1] = 2*Wo[1:nx-1]-Wm[1:nx-1]+C2*(Wo[2:nx]-2*Wo[1:nx-1]+Wo[0:nx-2])

    return Wm ## . . Return updated wavefield at time step n+1

Let's now build up a scenario that we are trying to model.  I will assume that the length of the region to be modeled is $L=1000$m, that is discretized into $nx=201$ (and thus $\Delta x = 5m$ segments), and the velocity is $v=1000$ m/s. We can also define the boundary conditions as $LB=RB=0$m.

In [3]:
## . . Define spatial grid
L  = 1000        # . . String length (m)
nx = 201      # . . Number of points in discretization 
dx = L/(nx-1) # . . Discretization interval
x = np.linspace(0,L,nx) # . . Xline
v  = 1000        # . . Velocity (m/s)

## . . Init wavefields on spatial grid
Up = np.zeros((nx))
Uo = np.zeros((nx))
Um = np.zeros((nx))

## . . Define BCs
LB = 0        # . . Left boundary (Dirchelet)
RB = 0        # . . Left boundary (Dirchelet)

Let's now define the time stepping conditions.  Because we have define our $\Delta x$ and $v$, we know that we need to obey the following Courant condition:

$$C \equiv \frac{v \Delta t}{\Delta x} \le 1$$

Thus, let's say that we choose $C=0.5$; thus, we can set 

$$ \Delta t = \frac{0.8\Delta x}{v} = \frac{0.8 \times 5m}{1000m/s} = 0.004 ms$$

We also want to define our initial conditions, which we will assume to be a Ricker wavelet:

$$\phi(x,t=0) = f(x) = \frac{2}{\sqrt{3\sqrt{\pi}}} \left(1 - ((x-L/2)/\sigma)^2\right)e^{-\frac{(x-L/2)^2}{2\sigma^2}}$$
$$\left.\frac{\partial \phi}{\partial t}\right|_{t=0} = g(x) = 0$$

where we will use $\sigma=40$ and set $\phi^{n-1}_i=0$.  Let's now create an animate function:

In [9]:
def AWE_1D_animate(i):
    global k
    amax = np.max(np.abs(c))
    ax1.clear()
    plt.plot(x,c[:,k],linewidth=3)
    plt.grid(True)
    plt.xlim([0,L])
    plt.ylim(-1.1*amax,1.1*amax)
    plt.xlabel('Distance (m)',fontsize=12);
    plt.ylabel('Amplitude',fontsize=12);
    plt.title('AWE at %f ms'%(k*dt*1000))
    k += 1

And then we can compute the solution:

In [10]:
## Time stepping parameters
CC = 0.5     # . . Courant #
nt = 800     # . . Number of time steps
dt = CC*dx/v # . . Define dt based on Courant

## . . Define Initial waveform
ss=40 ## sigma for Ricker wavelet
Uo = 2/np.sqrt(3*ss*np.sqrt(np.pi))*(1-((x-L/2     )/ss)**2)*np.exp(-(x-L/2     )**2/(2*ss**2))

## . . Total Solution space
c = np.zeros((nx,nt)) 

## . . Iterate over solution
for it in range(nt):
    tmp = awe_explicit_solver_homogeneous_2nd_order(Uo,Um,dx,dt,v,LB,RB) #calc solution at n+1
    c[:,it]=tmp  ## save solution vector
    Um=Uo        ## move solution at n to n-1 to prepare for next iteration
    Uo=tmp       ## move solution at n+1 to n to prepare for next iteration
    
## . . Animate solution
fig1 = plt.figure()
ax1 = fig1.add_subplot(1,1,1)
k = 0

# . . Call the animator. 
anim1 = animation.FuncAnimation(fig1,AWE_1D_animate,frames=nt-2,interval=25)

In [6]:
HTML(anim1.to_html5_video())

**Figure 10.1. Illustration of wave equation solution when one only defines the wavefield at $t=0$ and not $t=-\Delta t$.  Note that there is a two-way solution because there we haven't specified a constraint on one way or the other.**

Let's now look at what happens when we specify the previous wavefield state at $\phi(x,t=-\Delta t)$, which requires establishing what the wave looks like at the previous time step. Given the initial Ricker wavelet above, it is reasonable to expect that the $\phi(x,t=-\Delta t)$ will be given by:

$$\phi(x,t=-\Delta t) = \frac{2}{\sqrt{3\sqrt{\pi}}} \left(1 - ((x-L/2-v\Delta t )/\sigma)^2\right)e^{-\frac{(x-L/2-v\Delta t)^2}{2\sigma^2}}$$

where $v\Delta t$ has been introduced to account for a spatial shift.  Let's rerun the solution from above, but now where we reset the value of $\phi^{n-1}_i$: 

In [7]:
## Time stepping parameters
CC = 0.5     # . . Courant #
nt = 800     # . . Number of time steps
dt = CC*dx/v # . . Define dt based on Courant

## . . Define Initial waveform
ss=40 ## sigma for Ricker wavelet
Uo = 2/np.sqrt(3*ss*np.sqrt(np.pi))*(1-((x-L/2     )/ss)**2)*np.exp(-(x-L/2     )**2/(2*ss**2))

## . . Define previous time step (note depends on dt b/c need correct translation)
Um = 2/np.sqrt(3*ss*np.sqrt(np.pi))*(1-((x-L/2-dt*v)/ss)**2)*np.exp(-(x-L/2-dt*v)**2/(2*ss**2))

## . . Total Solution space
c = np.zeros((nx,nt)) 

## . . Iterate over solution
for it in range(nt):
    tmp = awe_explicit_solver_homogeneous_2nd_order(Uo,Um,dx,dt,v,LB,RB) #calc solution at n+1
    c[:,it]=tmp  ## save solution vector
    Um=Uo        ## move solution at n to n-1 to prepare for next iteration
    Uo=tmp       ## move solution at n+1 to n to prepare for next iteration
    
## . . Animate solution
fig1 = plt.figure()
ax1 = fig1.add_subplot(1,1,1)
k = 0

# . . Call the animator. 
anim1 = animation.FuncAnimation(fig1,AWE_1D_animate,frames=nt-2,interval=25)

In [8]:
HTML(anim1.to_html5_video())

**Figure 10.2. Illustration of waveequation solution when defining the wavefield at $t=0$ and $t=-\Delta t$.  Note that the wave initially travels to the left instead of both directions like in Figure 10.1 above.**

There is another parameter, $\sigma$, that we have set above that is related to the **frequency content** of the Ricker wavelet. Let's now investigate what happens when we set $\sigma=10$ instead of $\sigma=40$ above:

In [37]:
## Time stepping parameters
CC = 0.5     # . . Courant #
nt = 500     # . . Number of time steps
dt = CC*dx/v # . . Define dt based on Courant

## . . Define Initial waveform
ss=20 ## sigma for Ricker wavelet
Uo = 2/np.sqrt(3*ss*np.sqrt(np.pi))*(1-((x-L/2     )/ss)**2)*np.exp(-(x-L/2     )**2/(2*ss**2))

## . . Define previous time step (note depends on dt b/c need correct translation)
Um = 2/np.sqrt(3*ss*np.sqrt(np.pi))*(1-((x-L/2-dt*v)/ss)**2)*np.exp(-(x-L/2-dt*v)**2/(2*ss**2))

## . . Total Solution space
c = np.zeros((nx,nt)) 

## . . Iterate over solution
for it in range(nt):
    tmp = awe_explicit_solver_homogeneous_2nd_order(Uo,Um,dx,dt,v,LB,RB) #calc solution at n+1
    c[:,it]=tmp  ## save solution vector
    Um=Uo        ## move solution at n to n-1 to prepare for next iteration
    Uo=tmp       ## move solution at n+1 to n to prepare for next iteration
    
## . . Animate solution
fig1 = plt.figure()
ax1 = fig1.add_subplot(1,1,1)
k = 0

# . . Call the animator. 
anim1 = animation.FuncAnimation(fig1,AWE_1D_animate,frames=nt-2,interval=25)

In [ ]:
HTML(anim1.to_html5_video())

**Figure 10.3. Illustration of how numerical dispersion can degrade the solution over time.  The initial wave packet starts to develop a long "tail" that represents non-physical behaviour.**

## Higher-order derivatives

Let's now look at what happens if we use higher-order spatial derivatives.  I've rewritten the 1D solver from above to incorporate an $\mathcal{O}(\Delta x^8)$ approximation of the second spatial partial derivative.  This requires using a nine-point stencil that incorporates four points to the left and to the right of the stencil center

$$\phi^{n+1}_i = 2\phi^{n}_i -\phi^{n-1}_i +C^2 \left(-\frac{1}{560}\phi^{n}_{i+4}+\frac{8}{315}\phi^{n}_{i+3}-\frac{1}{5}\phi^{n}_{i+2}+\frac{8}{5}\phi^{n}_{i+1} -\frac{205}{72} \phi^{n}_i +\frac{8}{5} \phi^{n}_{i-1}-\frac{1}{5}\phi^{n}_{i-2}+\frac{8}{315}\phi^{n}_{i-3}-\frac{1}{560} \phi^{n}_{i-4}\right). \tag{8}$$

where these [finite-difference coefficients](https://en.wikipedia.org/wiki/Finite_difference_coefficient#Central_finite_difference) are for a **centered second derivative of eight order**. 

Note that we will again have issues at the boundaries where we will need **ghost points**. However, because we have longer stencil, we will now need to have four ghost points instead of just one.  Let's see how this work in our solver:

In [19]:
def awe_explicit_solver_homogeneous_8th_order(Wo,Wm,dx,dt,v,LB,RB):
    '''Set up eight-order solver of the acoustic wave equation
    usage: U = awe_explicit_solver_homogeneous(Wo,Wm,dx,dt,v,LB,RB)
    input: 
        Wo: Acoustic pressure vector (nx) at time step n
        Wm: Acoustic pressure vector (nx) at time step n-1
        dx: Spatial sampling
        dt: Temporal sampling
        v : Homogeneous propagation velocity 
        LB: Left  boundary (Homogeneous Dirchelet)
        RB: Right boundary (Homogeneous Dirchelet)
    output:
        Wm: Acoustic pressure vector (nx) at time n+1
    dependencies:
        None
    written by Jeff Shragge, jshragge@mines.edu, 10/2019
    '''        
    ## . . Get dimensions of wavefield
    nx = len(Wo)
    
    ## . . Define Courant number (squared)
    C2 = (v*dt/dx)**2
        
    ## . . Update solution 
    ## . . Note that I am overwriting Wm because this wavefield
    ## . . isn't needed after this calculation!
    Wm[4:nx-4] = 2*Wo[4:nx-4]-Wm[4:nx-4]+C2*( 
                    -1/560 *Wo[0:nx-8] 
                    +8/315 *Wo[1:nx-7]  
                    -1/5   *Wo[2:nx-6]  
                    +8/5   *Wo[3:nx-5]  
                    -205/72*Wo[4:nx-4]  
                    +8/5   *Wo[5:nx-3]  
                    -1/5   *Wo[6:nx-2]  
                    +8/315 *Wo[7:nx-1]  
                    -1/560 *Wo[8:nx  ])    
    
    return Wm ## . . Return updated wavefield at time step n+1

Let's now rerun our numerically dispersive solution from above with the $\mathcal{O}(\Delta x^8)$ solver:

In [36]:
## Time stepping parameters
CC = 0.5     # . . Courant #
nt = 500     # . . Number of time steps
dt = CC*dx/v # . . Define dt based on Courant

## . . Define Initial waveform
ss=20 ## sigma for Ricker wavelet
Uo = 2/np.sqrt(3*ss*np.sqrt(np.pi))*(1-((x-L/2     )/ss)**2)*np.exp(-(x-L/2     )**2/(2*ss**2))

## . . Define previous time step (note depends on dt b/c need correct translation)
Um = 2/np.sqrt(3*ss*np.sqrt(np.pi))*(1-((x-L/2-dt*v)/ss)**2)*np.exp(-(x-L/2-dt*v)**2/(2*ss**2))

## . . Total Solution space
d = np.zeros((nx,nt)) 

## . . Iterate over solution
for it in range(nt):
    tmp = awe_explicit_solver_homogeneous_8th_order(Uo,Um,dx,dt,v,LB,RB) #calc solution at n+1
    d[:,it]=tmp  ## save solution vector
    Um=Uo        ## move solution at n to n-1 to prepare for next iteration
    Uo=tmp       ## move solution at n+1 to n to prepare for next iteration
    
## . . Animate solution
fig1 = plt.figure()
ax1 = fig1.add_subplot(1,1,1)
k = 0

def AWE_1D_animate(i):
    global k
    amax = np.max(np.abs(d))
    ax1.clear()
    plt.plot(x,d[:,k],linewidth=3)
    plt.grid(True)
    plt.xlim([0,L])
    plt.ylim(-1.1*amax,1.1*amax)
    plt.xlabel('Distance (m)',fontsize=12);
    plt.ylabel('Amplitude',fontsize=12);
    plt.title('AWE at %f ms'%(k*dt*1000))
    k += 1
    
# . . Call the animator. 
anim1 = animation.FuncAnimation(fig1,AWE_1D_animate,frames=nt-2,interval=25)

In [35]:
HTML(anim1.to_html5_video())